STOCK DATA VISUALISATION TOOL

When the tool is run:

    - user is prompted to enter how many stocks they would like to see
    - user is then asked to enter the ticker symbols 
    - visualisations for stock price and stock volume over a 5 year range is displayed 


    - please see README.md

In [16]:
# import libraries 

import yfinance as yf
import pandas as pd
import plotly.express as px
import random 


In [17]:
# get the dataset and prepare it function

def get_and_prep_dataset(list_of_tickers, call_num):

    # get ticker symbol from list of tickers from user
    set_ticker = list_of_tickers[call_num]

    # get ticker from Yahoo Finance API
    ticker = yf.Ticker(set_ticker)

    # get historical data - 5yr range 
    from_yahoo = ticker.history(period = '5y')

    # set raw_dataset variable 
    raw_dataset = from_yahoo

    # series holding closing prices
    df_ser = raw_dataset['Close']
    
    # series holding volume
    df_volume_ser = raw_dataset['Volume']

    # convert series to pandas dataframes 
    df = pd.DataFrame(df_ser)
    df.index.name = None # fix for the two rows for column header issue
    df.reset_index(level = 0, inplace = True)
    df.rename(columns = {'index' : 'Date'}, inplace = True)

    df_volume = pd.DataFrame(df_volume_ser)
    df_volume.index.name = None # fix for the two rows for column header issue
    df_volume.reset_index(level = 0, inplace = True)
    df_volume.rename(columns = {'index' : 'Date'}, inplace = True)

    # set name variable from dictionary of company info
    name = ticker.info['longName']

    # set ticker symbol variable from dictionary of company info
    ticker_symbol = ticker.info['symbol']

    return df, df_volume, name, ticker_symbol, ticker


In [18]:
# information function 

def get_company_info(ticker):

    # information for ETFs
    if ticker.info['quoteType'] == 'ETF':
        print(' ')
        print('Stock: '  + ticker.info['longName'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')  
        print(' ')  
        print('Ticker Symbol: ' + ticker.info['symbol'])
        print('Type: Exchange Traded Fund (ETF)')
        print('Fund Family: ' + ticker.info['fundFamily'])
        print('Total Number of Assets: ' + str(ticker.info['totalAssets']))
        print(' ')
        print('Summary: ' + ticker.info['longBusinessSummary'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')   
        print(' ')
        print('Holdings: ' + str(ticker.info['holdings']))
        print('\n')

    # information for equities 
    elif ticker.info['quoteType'] == 'EQUITY':  
        print(' ') 
        print('Stock: '  + ticker.info['longName'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')  
        print(' ')
        print('Ticker Symbol: ' + ticker.info['symbol'])
        print('Industry: ' + ticker.info['industry'])
        print('Address: ' + ticker.info['address1'] + ', ' + ticker.info['city'] + ', ' + ticker.info['state'] + ', ' + ticker.info['country'])
        print(' ')
        print('Summary: ' + ticker.info['longBusinessSummary'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')  
        print(' ')
        print('Profit Margin: ' + str(ticker.info['profitMargins']))
        print('Total Revenue: $' + str(ticker.info['totalRevenue']))
        print('Total Cash: $' + str(ticker.info['totalCash']))
        print('Total Debt: $' + str(ticker.info['totalDebt']))
        print('Debt to Equity: ' + str(ticker.info['debtToEquity']))
        print('Operating Cashflow: $' + str(ticker.info['operatingCashflow']))
        print('52 Week Change: ' + str(ticker.info['52WeekChange']))
        print('\n')
    

In [19]:
# visualisation function

def show_visual(df, df_volume, name, ticker_symbol):
    
    # holds all colors that will be used 
    all_colors = ['blue', 'orange', 'red', 'green', 'purple', 'brown', 'violet', 'pink', 'darkgoldenrod', 'black', 'teal']

    # visualise stock price at closing
    fig = px.line(df, x = 'Date', y = 'Close', title = '<b>' + name + ' (' + ticker_symbol + ')' + '</b>' + ' - Stock Price', labels = {'value' : 'Close Price'})

    fig.update_layout(height = 575)
    fig.update_traces(hovertemplate = 'Date: %{x} <br>Close Price: $%{y}', line_color = random.choice(all_colors))

    fig.show()

    # visualise stock volume at closing
    fig_v = px.area(df_volume, x = 'Date', y = 'Volume', title = '<b>' + name + ' (' + ticker_symbol + ')' + '</b>' + ' - Stock Volume', labels = {'value' : 'Volume'})

    fig_v.update_layout(height = 575)
    fig_v.update_traces(hovertemplate = 'Date: %{x} <br>Volume: %{y}', line_color = random.choice(all_colors))

    fig_v.show()


In [20]:
# all calls and input requests for the user

# user enters number of stocks to see 
how_many = input('Enter how many stocks: ')

# creates an array that will hold user input for ticker symbols 
list_of_tickers = []
for h in range(int(how_many)):
    input_ticker = input('Enter ticker symbol: ')
    list_of_tickers.append(input_ticker.upper())

# presentation info
print('The tool will be run for ' + str(how_many) + ' stock/s')
print('The stock/s -> ' + str(list_of_tickers))
print('\n')

# function calls 
for call_num in range(len(list_of_tickers)):
    df, df_volume, name, ticker_symbol, ticker = get_and_prep_dataset(list_of_tickers = list_of_tickers, call_num = call_num)
    get_company_info(ticker)
    show_visual(df, df_volume, name, ticker_symbol)


The tool will be run for 3 stock/s
The stock/s -> ['TSLA', 'FNDX', 'FB']


 
Stock: Tesla, Inc.
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: TSLA
Industry: Auto Manufacturers
Address: 13101 Tesla Road, Austin, TX, United States
 
Summary: Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits. It provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; and purchase financing and leasing services. This segment is also involved in the provision of non-warranty after-sales vehicle services, sale of used vehicles, retail merchandise,

 
Stock: Schwab Fundamental U.S. Large Company Index ETF
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: FNDX
Type: Exchange Traded Fund (ETF)
Fund Family: Schwab ETFs
Total Number of Assets: 8869166080
 
Summary: The fund normally will invest at least 90% of its net assets (including, for this purpose, any borrowings for investment purposes) in stocks included in the index. The index measures the performance of the large company size segment by fundamental overall company scores (scores), which are created using as the universe the U.S. companies in the FTSE Global Total Cap Index (the parent index).
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Holdings: [{'symbol': 'AAPL', 'holdingName': 'Apple Inc', 'holdingPercent': 0.041500002}, {'symbol': 'XOM', 'holdingName': 'Exxon Mobil Corp', 'holdingPercent': 0.0219}, {'symbol': 'MSFT', 'hold

 
Stock: Meta Platforms, Inc.
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: FB
Industry: Internet Content & Information
Address: 1601 Willow Road, Menlo Park, CA, United States
 
Summary: Meta Platforms, Inc. develops products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, wearables, and in-home devices worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment's products include Facebook, which enables people to share, discover, and connect with interests; Instagram, a community for sharing photos, videos, and private messages, as well as feed, stories, reels, video, live, and shops; Messenger, a messaging application for people to connect with friends, family, groups, and businesses across platforms and devices through chat, audio and video calls, and rooms; and WhatsApp, a m